2Bilayer unit domain. Sigma optimisation. Erro eig+eigv.

In [1]:
# Importing libraries
import matplotlib.pyplot as plt
from matplotlib import ticker
import numpy as np
from mpi4py import MPI
from ufl import (
    inner,
    grad,
    conj,
    TrialFunction,
    TestFunction,
    as_matrix,
    as_vector,
    dot,
    Or,
    Measure,
    conditional,
    SpatialCoordinate,
)
from dolfinx.mesh import locate_entities_boundary, create_interval, locate_entities, create_submesh
from dolfinx.fem import FunctionSpace, Function, functionspace, Function, dirichletbc, locate_dofs_topological, petsc, form, assemble_matrix
import basix
from petsc4py.PETSc import ScalarType
from slepc4py.SLEPc import EPS  
colors_list = ["blue", "green", "red", "magenta", "black", "purple", "orange", "brown", "pink"]


/home/srecondo/miniconda3/envs/cleaned-fenicsx/lib/python3.10/site-packages/numpy/_core/getlimits.py:551: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


Domain, PML, refinement and physical configurations

In [2]:
# Define element size for the mesh refinement
h_i = [0.125,
    0.0625,
    0.0625 / 2,
    0.0625 / 4,
    0.0625 / 8]

# Domain configuration
Lx = 1. # fluid domain length (without PML)
l_inner=Lx/2

# Physical configuration
rho_media_1 = 1. # kg/m^3 
vel_media_1 = 1.   # m/s
rho_media_2 = 1. # kg/m^3 
vel_media_2 = 2.   # m/s


# Define the density and velocity functions  
def rho_function(x):    
    return np.where(x[0] > l_inner, rho_media_2,rho_media_1)

def vel_function(x):    
    return np.where(x[0] > l_inner, vel_media_2,vel_media_1)

# PML configuration
pml_thickness = Lx/4
sigma_0 = 1 #0.484375

# Solver configuration

requested_eigenvalues_number = 50 # Number of eigenvalues to compute

Solver configuration and computing

In [3]:
def get_configured_problem(mesh, degrees, sigma_x):
    # Initialize dictionaries
    dof = {}
    solver_dict={}
    V_dict={}
    Q_dict={}

    # Define finite element spaces
    p_fe_type = basix.ufl.element(family="Lagrange", cell=mesh.basix_cell(), degree=degrees[0])
    v_fe_type = basix.ufl.element(family="DG", cell=mesh.basix_cell(), degree=degrees[1], shape=(mesh.geometry.dim,))
    FE = {"p": p_fe_type, "v": v_fe_type}

    # Create a submesh of PML associated cells
    tdim = mesh.topology.dim

    # Identify the fluid cells
    fluid_cells = locate_entities(mesh, tdim, lambda x: x[0] <= Lx)

    # Create function spaces for mesh and submesh
    Q = functionspace(mesh,FE["p"])
    V = functionspace(mesh, FE["v"])

    # Dictionaries to store the number of dofs
    dof["p"] = Q.dofmap.index_map.size_global
    dof["v"] = V.dofmap.index_map.size_global

    # Boundary conditions application (for pressure)
    bc = []
    facets = locate_entities_boundary(mesh, 0, lambda x: np.isclose(x[0], 0.0))
    o_boundary_dofs = locate_dofs_topological(Q, 0, facets)  # the dimension is related to mesh entity where dofs are located

    # Setting to zero located dofs related to boundaries
    u_d = Function(Q)
    u_d.x.array[:] = 0.0  # Asign Dirichlet condition value
    bc.append(dirichletbc(u_d, o_boundary_dofs)) 
    
    facets = locate_entities_boundary(mesh, 0, lambda x: np.isclose(x[0], Lx+pml_thickness))
    o_boundary_dofs = locate_dofs_topological(Q, 0, facets)  # the dimension is related to mesh entity where dofs are located

    # Setting to zero located dofs related to boundaries
    u_d = Function(Q)
    u_d.x.array[:] = 0.0  # Asign Dirichlet condition value
    bc.append(dirichletbc(u_d, o_boundary_dofs)) 

    # Define differential operators
    dx = Measure("dx", domain=mesh, subdomain_data=[(1, fluid_cells)])

    # Interpolates the density and velocity functions
    W = functionspace(mesh, ("DG", 2))
    rho = Function(W)
    rho.interpolate(rho_function)
    c = Function(W)
    c.interpolate(vel_function)

    # Spatial Coordinate is updated for the actual mesh
    x = SpatialCoordinate(mesh)

    # Variation formulation definition
    p = TrialFunction(Q) 
    v = TrialFunction(V)
    q = TestFunction(Q)
    w = TestFunction(V)

    # Define the variational forms
    a = form(
        [
            [sigma_x(x) / (rho * c**2) * p * conj(q) * dx, -1 * inner(v, grad(q)) * dx],
            [inner(grad(p), w) * dx, sigma_x(x) * rho * inner(v, w) * dx],
        ]
    )
    b = form(
        [
            [1.0 / (rho * c**2) * p * conj(q) * dx, None],
            [None, rho * inner(v, w) * dx],
        ]  
    )
    # Assemble matrices
    A = petsc.assemble_matrix_block(a, bcs=bc)
    A.assemble()
    B = petsc.assemble_matrix_block(b, bcs=bc)
    B.assemble()

    # SLEPc solver configuration for linear eigenvalue problem (A x =\lambda x B)
    solver = EPS().create()
    solver.setOperators(-A, B)
    solver.setType("arnoldi")
    solver.setWhichEigenpairs(9)  # TARGET_IMAGINARY = 9, TARGET_MAGNITUDE = 7, TARGET_REAL = 8
    solver.setDimensions(
        requested_eigenvalues_number,
        4 * requested_eigenvalues_number,
        4 * requested_eigenvalues_number,
    )
    solver.setTolerances(1e-7, 100)
    st = solver.getST()
    st.setType("sinvert")  # Applying spectral transformation "shift and invert" 
    solver.setTarget(-2.5j)  
    solver.solve()
    n_eigs_converged_quadratic = solver.getConverged()

    return solver, mesh, Q, V, dof

Approx eigenvalues calculation

In [4]:
def get_approx_eigenpair(solver, selected_index, Q, V, dof):
    # Gets p, v eigenvectors
    p_h = Function(Q, name="p")
    v_h = Function(V, name="v")
    dof["p"] = Q.dofmap.index_map.size_global
    dof["v"] = V.dofmap.index_map.size_global
    [vrA, viA] = solver.getOperators()[0].getVecs()  # type: ignore

    # Filtrar eigenvalores con parte real mayor que 1e-12
    eigenvalues = []
    for i in range(solver.getConverged()):
        omega_value = solver.getEigenpair(i, vrA) * (1j)  # type: ignore
        if omega_value.real > 1e-12:
            eigenvalues.append((omega_value, vrA.copy()))

    # Seleccionar el n-ésimo eigenvalor filtrado
    omega_value, vrA = eigenvalues[selected_index]

    # FEM approximation for the eigenfunction
    p_h.x.petsc_vec.array[:] = vrA[0 : dof["p"]]
    p_h.x.petsc_vec.array[:] = p_h.x.petsc_vec.array[:] / p_h.eval([Lx, 0.0, 0.0], 1)  # normalise
    v_h.x.petsc_vec.array[:] = vrA[dof["p"] :]  
    v_h.x.petsc_vec.array[:] = v_h.x.petsc_vec.array[:] / v_h.eval([Lx, 0.0, 0.0], 1)  # normalise

    return [omega_value, p_h, v_h]  # Return eigenfunction and eigenvalue

In [5]:
def get_exact_eigenpair(solver, selected_index, Q, V):
    exact_physical_eigs_function= lambda n: vel_media_1 / l_inner * np.arctan(-1j * (vel_media_2 * rho_media_2) / (vel_media_1 * rho_media_1))+ n * np.pi * vel_media_1 / l_inner
    w=exact_physical_eigs_function(selected_index)

    p_1 = lambda x: np.sin(w * x[0] / vel_media_1)
    p_2 = lambda x: -np.sin(-w * l_inner / vel_media_1) * np.exp(1j * w / vel_media_2 * (x_hat(x,w) - l_inner))
    p = lambda x: np.where(x[0] == Lx + pml_thickness, 0, np.where(x[0] >= l_inner, p_2(x), p_1(x)))

    sigma_x = lambda x: np.where(x[0] >= l_inner, sigma_0, 0)
    sigma_x_function_cond= lambda x: np.where(x[0]<=Lx, x[0]*0., np.where(x[0] == Lx + pml_thickness, x[0]*0.0+1e10, sigma_x(x)))
    v_1=lambda x: 1/(-1j*w)*(w / vel_media_1) * np.cos(w * x[0] / vel_media_1)
    v_2 = (lambda x: 1/(-1j*w+sigma_x_function_cond(x)*rho_media_1)* -np.sin(-w * l_inner / vel_media_1) *(1j * w / vel_media_2)
        * np.exp(1j * w / vel_media_2 * (x_hat(x,w) - l_inner))* gamma_x(x, w)[0])
    v = lambda x: np.where(x[0] >= l_inner, v_2(x), v_1(x))

    Q = functionspace(mesh, FE["p"])
    V = functionspace(mesh, FE["v"])

    p_ex = Function(Q, name="p_ex")
    p_ex.interpolate(p)
    p_ex.x.petsc_vec.array[:] = p_ex.x.petsc_vec.array[:] / p_ex.eval(np.array([Lx, 0.0, 0.0]), 1)

    v_ex = Function(V, name="v_ex")
    v_ex.interpolate(v)
    v_ex.x.petsc_vec.array[:] = v_ex.x.petsc_vec.array[:] / v_ex.eval(np.array([Lx, 0.0, 0.0]), 1)

    return [w, p_ex, v_ex]

quadratic

In [6]:
# Define quadratic sigma function
sigma_x_quadratic=lambda x: conditional(x[0] > Lx, sigma_0*(x[0]-Lx)**2, 0.0)
# Initialize lists to store the approximated eigenpairs
approx_quadratic_P1_P0 = [[0 for _ in range(5)] for _ in range(len(h_i))]
approx_quadratic_P2_P1 = [[0 for _ in range(5)] for _ in range(len(h_i))]
approx_quadratic_P3_P2 = [[0 for _ in range(5)] for _ in range(len(h_i))]
approx_quadratic_P4_P3 = [[0 for _ in range(5)] for _ in range(len(h_i))]

for i,h in enumerate(h_i):
    # Create mesh depending on h
    mesh = create_interval(MPI.COMM_WORLD, int((Lx + pml_thickness )/ h), [0, Lx + pml_thickness])
    # Define solver for quadratic PML
    [solver_quadratic_P1_P0, mesh, Q_P1_P0, V_P1_P0, dof_P1_P0]=get_configured_problem(mesh, [1,0], sigma_x_quadratic)
    [solver_quadratic_P2_P1, mesh, Q_P2_P1, V_P2_P1, dof_P2_P1]=get_configured_problem(mesh, [2,1], sigma_x_quadratic)
    [solver_quadratic_P3_P2, mesh, Q_P3_P2, V_P3_P2, dof_P3_P2]=get_configured_problem(mesh, [3,2], sigma_x_quadratic)
    [solver_quadratic_P4_P3, mesh, Q_P4_P3, V_P4_P3, dof_P4_P3]=get_configured_problem(mesh, [4,3], sigma_x_quadratic)

    for j in np.arange(0, 5,1):
        # Get the approximated eigenpair
        approx_quadratic_P1_P0[i][j] = get_approx_eigenpair(solver_quadratic_P1_P0, j, Q_P1_P0, V_P1_P0, dof_P1_P0)[0]
        approx_quadratic_P2_P1[i][j] = get_approx_eigenpair(solver_quadratic_P2_P1, j, Q_P2_P1, V_P2_P1, dof_P2_P1)[0]
        approx_quadratic_P3_P2[i][j] = get_approx_eigenpair(solver_quadratic_P3_P2, j, Q_P3_P2, V_P3_P2, dof_P3_P2)[0]
        approx_quadratic_P4_P3[i][j] = get_approx_eigenpair(solver_quadratic_P4_P3, j, Q_P4_P3, V_P4_P3, dof_P4_P3)[0]

# Ordenar cada fila de la lista
for i in range(len(approx_quadratic_P1_P0)):
    approx_quadratic_P1_P0[i] = sorted(approx_quadratic_P1_P0[i], key=lambda x: x.real)
    approx_quadratic_P2_P1[i] = sorted(approx_quadratic_P2_P1[i], key=lambda x: x.real)
    approx_quadratic_P3_P2[i] = sorted(approx_quadratic_P3_P2[i], key=lambda x: x.real)
    approx_quadratic_P4_P3[i] = sorted(approx_quadratic_P4_P3[i], key=lambda x: x.real)


In [7]:
sigma_quadratic_absorption=(Lx+pml_thickness)**3/3-Lx*(Lx+pml_thickness)**2+Lx**2*(Lx+pml_thickness)-sigma_0*Lx**3/3
sigma_piecewise_absorption=sigma_0*pml_thickness
sigma_singular_absorption=1e40

In [8]:
def group_by_real_part(eigenvalues, tolerance=1):
    groups = []
    for row in eigenvalues:
        for eig in row:
            added = False
            for group in groups:
                if abs(group[0].real - eig.real) < tolerance:
                    group.append(eig)
                    added = True
                    break
            if not added:
                groups.append([eig])
    return groups

groups_P1_P0 = group_by_real_part(approx_quadratic_P1_P0)
groups_P2_P1 = group_by_real_part(approx_quadratic_P2_P1)
groups_P3_P2 = group_by_real_part(approx_quadratic_P3_P2)
groups_P4_P3 = group_by_real_part(approx_quadratic_P4_P3)

In [9]:
rel_error_eig_0_P1_P0=abs((groups_P1_P0[0] - exact[1])) / np.abs(exact[1]) 
rel_error_eig_1_P1_P0=abs((groups_P1_P0[1] - exact[2])) / np.abs(exact[2]) 
rel_error_eig_2_P1_P0=abs((groups_P1_P0[2] - exact[3])) / np.abs(exact[3]) 
rel_error_eig_3_P1_P0=abs((groups_P1_P0[3] - exact[4])) / np.abs(exact[4]) 

rel_error_eig_0_P2_P1=abs((groups_P2_P1[0] - exact[1])) / np.abs(exact[1]) 
rel_error_eig_1_P2_P1=abs((groups_P2_P1[1] - exact[2])) / np.abs(exact[2]) 
rel_error_eig_2_P2_P1=abs((groups_P2_P1[2] - exact[3])) / np.abs(exact[3]) 
rel_error_eig_3_P2_P1=abs((groups_P2_P1[3] - exact[4])) / np.abs(exact[4]) 

rel_error_eig_0_P3_P2=abs((groups_P3_P2[0] - exact[1])) / np.abs(exact[1]) 
rel_error_eig_1_P3_P2=abs((groups_P3_P2[1] - exact[2])) / np.abs(exact[2]) 
rel_error_eig_2_P3_P2=abs((groups_P3_P2[2] - exact[3])) / np.abs(exact[3]) 
rel_error_eig_3_P3_P2=abs((groups_P3_P2[3] - exact[4])) / np.abs(exact[4]) 

rel_error_eig_0_P4_P2=abs((groups_P4_P3[0] - exact[1])) / np.abs(exact[1]) 
rel_error_eig_1_P4_P2=abs((groups_P4_P3[1] - exact[2])) / np.abs(exact[2]) 
rel_error_eig_2_P4_P2=abs((groups_P4_P3[2] - exact[3])) / np.abs(exact[3]) 
rel_error_eig_3_P4_P2=abs((groups_P4_P3[3] - exact[4])) / np.abs(exact[4]) 

NameError: name 'exact' is not defined